In [1]:
!pip install python-docx nltk matplotlib pandas replicate seaborn xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
# r8_52F6ybHAUMB18XMdvCdbEZsC8SUGgS8161dCy

In [2]:
# Importación de bibliotecas
import docx
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
import replicate
import xlsxwriter

# Descargar stopwords en español
nltk.download('stopwords')

# Función para leer el documento Word
def leer_documento_word(ruta_archivo):
    doc = docx.Document(ruta_archivo)
    return ' '.join([p.text for p in doc.paragraphs])

# Función para extraer competencias con LLM
def extraer_competencias(api_key, modelo, texto):
    prompt = f"""Analiza el texto y lista las competencias profesionales más relevantes:

    {texto[:10000]}

    Formato requerido:
    1. Competencia 1
    2. Competencia 2
    ..."""

    try:
        cliente = replicate.Client(api_token=api_key)
        resultado = cliente.run(modelo, input={"prompt": prompt})
        return re.findall(r'\d+\.\s+(.+?)(?=\d+\.|$)', ''.join(resultado), re.DOTALL)
    except Exception as e:
        print(f"Error: {e}")
        return []

# Función para calcular frecuencias mejorada
def calcular_frecuencias(texto, competencias):
    texto_limpio = re.sub(r'[^\w\s]', '', texto.lower())
    frecuencias = {}

    for competencia in competencias:
        palabras_clave = [p for p in competencia.lower().split()
                        if p not in stopwords.words('spanish') and len(p) > 3]
        count = 0
        for palabra in palabras_clave:
            count += texto_limpio.count(palabra)
        frecuencias[competencia] = count if count > 0 else 1  # Evitar ceros

    return dict(sorted(frecuencias.items(), key=lambda x: x[1], reverse=True))

# Función para mostrar tabla detallada
def mostrar_tabla(frecuencias):
    df = pd.DataFrame(frecuencias.items(), columns=['Competencia', 'Frecuencia'])
    df['% Importancia'] = (df['Frecuencia'] / df['Frecuencia'].sum() * 100).round(2)
    df['Nivel de Prioridad'] = pd.cut(df['% Importancia'],
                                    bins=3,
                                    labels=['Alta', 'Media', 'Baja'])

    print("\nTabla de Competencias Prioritarias:")
    print(df.to_string(index=False))

    # Guardar en Excel con formato
    writer = pd.ExcelWriter('competencias_prioritarias.xlsx', engine='xlsxwriter')
    df.to_excel(writer, index=False)

    # Formatear el archivo Excel
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Formato para porcentajes
    percent_format = workbook.add_format({'num_format': '0.00%'})
    worksheet.set_column('C:C', 12, percent_format)

    # Formato condicional
    worksheet.conditional_format('B2:B16', {
        'type': 'data_bar',
        'bar_color': '#63C384'
    })

    writer.close()

# Función principal simplificada
def main():
    texto = leer_documento_word("/content/Gastron_internacional_CFT-ST.docx")
    competencias = extraer_competencias("r8_52F6ybHAUMB18XMdvCdbEZsC8SUGgS8161dCy",
                                      "meta/meta-llama-3-70b-instruct",
                                      texto)

    if competencias:
        frecuencias = calcular_frecuencias(texto, competencias)
        mostrar_tabla(frecuencias)
    else:
        print("No se encontraron competencias")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



Tabla de Competencias Prioritarias:
                                                                                                                                                                                                                           Competencia  Frecuencia  % Importancia Nivel de Prioridad
                                        **Comunicación Efectiva**: Comunicar en forma verbal y no verbal, utilizando estrategias de habilidades comunicativas para expresar aspectos del quehacer social, académico y profesional.\n\n          19          16.10               Baja
**Comunicación en Idiomas**: Utilizar frases y expresiones relacionadas con su área de experiencia destinadas a comunicarse de manera efectiva en un idioma extranjero (en este caso, nivel A1 del Marco Común Europeo de Referencia).          19          16.10               Baja
                          **Análisis y Resolución de Problemas**: Utilizar las herramientas básicas de las ciencias para la represen